In [29]:
import pandas as pd
import numpy as np
transfusion = pd.read_csv(r"C:\Users\ekta hinduja\Downloads\transfusion\transfusion.data")
transfusion


,Recency (months),Frequency (times),Monetary (c.c. blood),Time (months),whether he/she donated blood in March 2007
0,2,50,12500,98,1
1,0,13,3250,28,1
2,1,16,4000,35,1
3,2,20,5000,45,1
4,1,24,6000,77,0
...,...,...,...,...,...
743,23,2,500,38,0
744,21,2,500,52,0
745,23,3,750,62,0
746,39,1,250,39,0


In [30]:
transfusion

,Recency (months),Frequency (times),Monetary (c.c. blood),Time (months),whether he/she donated blood in March 2007
0,2,50,12500,98,1
1,0,13,3250,28,1
2,1,16,4000,35,1
3,2,20,5000,45,1
4,1,24,6000,77,0
...,...,...,...,...,...
743,23,2,500,38,0
744,21,2,500,52,0
745,23,3,750,62,0
746,39,1,250,39,0


In [31]:
transfusion.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 748 entries, 0 to 747
Data columns (total 5 columns):
Recency (months)                              748 non-null int64
Frequency (times)                             748 non-null int64
Monetary (c.c. blood)                         748 non-null int64
Time (months)                                 748 non-null int64
whether he/she donated blood in March 2007    748 non-null int64
dtypes: int64(5)
memory usage: 29.3 KB


In [32]:
transfusion.rename(columns={'whether he/she donated blood in March 2007': 'target'},inplace=True)
transfusion.head()


,Recency (months),Frequency (times),Monetary (c.c. blood),Time (months),target
0,2,50,12500,98,1
1,0,13,3250,28,1
2,1,16,4000,35,1
3,2,20,5000,45,1
4,1,24,6000,77,0


In [33]:
transfusion.target.value_counts(normalize=True).round(2)


0    0.76
1    0.24
Name: target, dtype: float64

In [34]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    transfusion.drop(columns='target'),
    transfusion.target,
    test_size=0.25,
    random_state=42,
    stratify=transfusion.target
)

X_train.head(2)


,Recency (months),Frequency (times),Monetary (c.c. blood),Time (months)
334,16,2,500,16
99,5,7,1750,26


In [35]:
!pip install TPOT


In [37]:
from tpot import TPOTClassifier
from sklearn.metrics import roc_auc_score

tpot = TPOTClassifier(
    generations=5,
    population_size=20,
    verbosity=2,
    scoring='roc_auc',
    random_state=42,
    disable_update_check=True,
    config_dict='TPOT light'
)
tpot.fit(X_train, y_train)

tpot_auc_score = roc_auc_score(y_test, tpot.predict_proba(X_test)[:, 1])
print(f'\nAUC score: {tpot_auc_score:.4f}')

print('\nBest pipeline steps:', end='\n')
for idx, (name, transform) in enumerate(tpot.fitted_pipeline_.steps, start=1):
    
    print(f'{idx}. {transform}')


ImportError: cannot import name '_safe_indexing' from 'sklearn.utils' (C:\Users\ekta hinduja\Anaconda3\lib\site-packages\sklearn\utils\__init__.py)

In [38]:
X_train.var().round(2)


Recency (months)              66.93
Frequency (times)             33.83
Monetary (c.c. blood)    2114363.70
Time (months)                611.15
dtype: float64

In [39]:
X_train_normed, X_test_normed = X_train.copy(), X_test.copy()
col_to_normalize = 'Monetary (c.c. blood)'


for df_ in [X_train_normed, X_test_normed]:
    df_['Monetary_log'] = np.log(df_[col_to_normalize])
    df_.drop(columns=col_to_normalize, inplace=True)

X_train_normed.var().round(2)


Recency (months)      66.93
Frequency (times)     33.83
Time (months)        611.15
Monetary_log           0.84
dtype: float64

In [40]:
from sklearn import linear_model

logreg = linear_model.LogisticRegression(C=0.1, dual=False, penalty='l2',solver='liblinear',random_state=42)

# Train the model
logreg.fit(X_train_normed, y_train)


logreg_auc_score = roc_auc_score(y_test, logreg.predict_proba(X_test_normed)[:, 1])
print(f'\nAUC score: {logreg_auc_score:.4f}')


ImportError: cannot import name '_deprecate_positional_args' from 'sklearn.utils.validation' (C:\Users\ekta hinduja\Anaconda3\lib\site-packages\sklearn\utils\validation.py)